### 결측치

- NaN : 값이 있어야 하는데 없는 결측 대체, 추정, 예측
- None : 값이 없는게 값이 결측 (ex. 직업-없음) 새로운 값으로 정의하는 방식으로 처리
- 결측치 처리 방법은 매우 간단
    - 상황(도메인)에 따라 다른 처리방법 선택인 매우 중요


#### 용어
- 결측 레코드 : 결측치를 포함하는 레코드
- 결측치 비율 : 결측 레코드 수 / 전체 레코드 개수

#### 결측치 삭제
- 행단위 결측 삭제
    - 결측이 없는 레코드가 모델 학습는데 충분해야 함 - 학습 샘플 개수에 따른 서능의 수렴여부로 확인 결정
    - 결측이 새로 들어올 데이터에는 없어야 함 - 도메인 지식 기반의 판단이 필요함
    
    
- 열단위 결측 삭제
    - 어느 특정 변수에만 결측이 많이 포함되어 있음
    - 해당 변수들이 크게 중요하지 않음 (도메인 지식)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

#그래프 패키지 
import matplotlib.pyplot as plt 
%matplotlib inline 
# 한글 문제 해결
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')# 한글 문제 해결
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')


# 동시 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
# 데이터 로드
df = pd.read_csv('./data/mammographic.csv')

In [3]:
df.shape
df.head()
df.tail()

(961, 6)

,BI-RADS,Age,Shape,Margin,Density,Output
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


,BI-RADS,Age,Shape,Margin,Density,Output
956,4.0,47.0,2.0,1.0,3.0,0
957,4.0,56.0,4.0,5.0,3.0,1
958,4.0,64.0,4.0,5.0,3.0,0
959,5.0,66.0,4.0,5.0,3.0,1
960,4.0,62.0,3.0,3.0,3.0,0


In [7]:
# 특징과 레이블 분리
X = df.drop('Output', axis=1) # 열삭제
Y = df['Output']

In [10]:
# 학습 데이터와 평가 데이터 분할
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

### 학습 데이터만의 전처리

In [11]:
# 결측치 확인 - 컬럼별 결측치 개수 확인
Train_X.isnull().sum(axis=0)

BI-RADS     2
Age         3
Shape      23
Margin     32
Density    61
dtype: int64

In [14]:
# 열별 결측 비율 확인
Train_X.isnull().sum(axis=0)/len(Train_X)

BI-RADS    0.002778
Age        0.004167
Shape      0.031944
Margin     0.044444
Density    0.084722
dtype: float64

In [15]:
# 결측 비율이 높지 않으므로 결측이 포함된 레코드 제거하기로 결정
Train_X.dropna(inplace=True)
# 경고 : 새로 들어온 레코드의 결측이 있ㅇ드면 처리하지 못함을 의미

<ipython-input-15-79b4b0535c11>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Train_X.dropna(inplace=True)


In [16]:
Train_X.shape # 레코드 삭제 조건 1 : 삭제후 충분한 데이터가 있어야 함

(619, 5)

In [17]:
# 새로수집되었다고 가정한 Test_X에 결측치가 포함되어 있음 : 레코드 삭제 조건 2을 위배
Test_X.isnull().sum(axis=0)

BI-RADS     0
Age         2
Shape       8
Margin     16
Density    15
dtype: int64

### 열단위 삭제

In [18]:
df = pd.read_csv('./data/post_operative.csv')

In [19]:
df.shape
df.head()
df.tail()

(90, 9)

,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,Decision
0,mid,low,excellent,mid,stable,stable,stable,15,A
1,mid,high,excellent,high,stable,stable,stable,?,S
2,high,low,excellent,high,stable,stable,mod-stable,?,A
3,mid,low,good,high,stable,unstable,mod-stable,15,A
4,mid,mid,excellent,high,stable,stable,stable,?,A


,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT,Decision
85,mid,mid,excellent,mid,unstable,stable,stable,10,A
86,mid,mid,excellent,mid,unstable,stable,stable,15,S
87,mid,mid,good,mid,unstable,stable,stable,15,A
88,mid,mid,excellent,mid,unstable,stable,stable,?,A
89,mid,mid,good,mid,unstable,stable,stable,15,S


In [20]:
# 특징과 라벨 분리
X = df.drop('Decision', axis = 1)
Y = df['Decision']

In [21]:
# 학습 데이터와 평가 데이터 분할
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [22]:
Train_X.head() # COMFORT 컬럼에 ? 값이 있고 결측이라고 전달 받음 - 도메인 지식에 기반해야 함

,L-CORE,L-SURF,L-O2,L-BP,SURF-STBL,CORE-STBL,BP-STBL,COMFORT
71,low,low,excellent,mid,stable,stable,stable,10
51,low,high,excellent,mid,unstable,stable,stable,?
38,low,low,good,mid,stable,stable,stable,?
65,mid,mid,good,high,stable,stable,stable,?
12,high,high,excellent,high,unstable,stable,unstable,15


In [23]:
# 결측 비율을 확인
Train_X.replace('?', np.nan).isnull().sum() / len(Train_X)

L-CORE       0.000000
L-SURF       0.000000
L-O2         0.000000
L-BP         0.000000
SURF-STBL    0.000000
CORE-STBL    0.000000
BP-STBL      0.000000
COMFORT      0.298507
dtype: float64

In [24]:
# 결측은 COMFORT 에 쏠려 있고, 해당 변순느 중요하지 않다는 도메인 지식 기반하에 삭제 가능

Train_X.drop('COMFORT', axis=1, inplace=True)

# 새로 수집되는 데이터에도 결측 발생을  용인(COMFORT르 아예  수집하지 않게 처리? - 도메인 기반해서 결정)
# 새로 수집된 데이터에 있는 결측 컬럼도 제거해서 서비스에 사용해야 함
Test_X.drop('COMFORT', axis=1,  inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [25]:
Train_X.isnull().sum(axis = 0)

L-CORE       0
L-SURF       0
L-O2         0
L-BP         0
SURF-STBL    0
CORE-STBL    0
BP-STBL      0
dtype: int64